In [ ]:
from bs4 import BeautifulSoup as bs
import requests 
import numpy as np
import pandas as pd
import re #Using regular expressions to search for special sequence of characters
import winsound
import time

In [ ]:

#These 2 arrays are used to store the webpages extracted through webscraping
IndustryLinks=np.empty(0)
CompaniesLinks=np.empty(0)

#This array stores the part of the name of industry used as keyword while parsing through html doc
IndustryNames=np.empty(0)

#These 2 arrays are used to store the names of industries and companies extracted
CompanyNames=np.empty(0)
AllIndustries=np.empty(0)

#Used to store the values of parameters that is extracted for each company

beta=np.empty(0)#contains beta values
pe=np.empty(0)#contains p/e values
div=np.empty(0)#contains dividend values
indpe=np.empty(0)#contains industry p/e values
pb=np.empty(0)#contains p/b values
divy=np.empty(0)#contains dividend yield values
deliv1d=np.empty(0)#contains 1-day deliverables
deliv3d=np.empty(0)#contains 3-day deliverables
deliv5d=np.empty(0)#contains 5-day deliverables
deliv8d=np.empty(0)#contains 8-day deliverables

df=pd.DataFrame(columns=['Industry','Company','Beta','P/E','Dividend(%)','Industry P/E','P/B','Dividend Yield(%)','Deliverables(1D)','Deliverables(3D)','Deliverables(5D)','Deliverables(8D)'])

In [ ]:
def Beta(Clink):
    
    page=requests.get(Clink)       #Using companylink passed tocreate soup object
    soup=bs(page.content,'lxml')
    
    attr4={'class':'nsert','id':'div_nse_livebox_wrap'}           #Using attributes to locate nse section of 
    nseData=soup.find('div',attrs=attr4)                          #html doc
    beta=nseData.find_all('div',attrs={'class':'disin vt'})       #Finding the part of the code 
    
    for div in beta:
        temp=str([div.text])
        
    temp1=temp.split('\\n')
    #print(temp1[2])
    try:
        return float(temp1[2])
    except:
        return np.NaN

In [ ]:
def terms(Clink):
    try:
        temp=np.empty(0)
        temp1=np.empty(0)
        page=requests.get(Clink)
        soup=bs(page.content,'lxml')
        attr={'class':'tab-pane fade','id':'consolidated_valuation','role':'tabpanel'}
        div=soup.find('div',attrs=attr)
        data=div.find_all('div',attrs={'class':'value_txtfr'})
        for t in data:     
            temp=np.append(temp,[t.text])
        temp1=temp[11].split()
        values=np.array([temp[1],temp[3],temp[5],temp[8],temp[9],temp1[9],temp1[4],temp1[6],temp1[8]])
        series=[]
        for i in values:
            try:
                series.append(float(i))
            except(ValueError):
                series.append(np.NaN)
        return series
    except:
        series=[]
        for i in range(0,9):
            series.append(np.NaN)
        return series

In [ ]:
page=requests.get("https://www.moneycontrol.com/stocks/marketstats/industry-classification/nse/abrasives.html")
soup=bs(page.content,'lxml') #creating soup object to access links of all the industries

attr={'class':'colLft'}
section=soup.find('section',attrs=attr)#Scanning for section that contains list of all links


attr2={'href':re.compile("/stocks/marketstats/industry-classification/nse/")}
links=section.find_all('a',attrs=attr2)#storing all href containing links after parsing isdone

IndustryLinks=np.append(IndustryLinks,'https://www.moneycontrol.com/stocks/marketstats/industry-classification/nse/abrasives.html')
link1='https://www.moneycontrol.com'

In [ ]:
i=1
for link in links:
    IndustryLinks=np.append(IndustryLinks,link1+str(link.get('href')))
    
    i=i+1
#Extracting all industry links from html doc and storing them in numpy array

In [ ]:
i=0
for link in IndustryLinks:
    try:
        page=requests.get(link)
        soup=bs(page.content,'lxml')
        attr={'border':'0','cellpadding':'0','cellspacing':'0','width':'100%'}
        table=soup.find('table',attrs=attr)
        tbody=table.find('tbody')
        Complinks=tbody.find_all('a',attrs={'href':re.compile("https://www.moneycontrol.com/india/stockpricequote/")})
        for j in Complinks:
            temp=str(j).split("href=\"")
            temp1=temp[1].split("\" style")
            CompaniesLinks=np.append(CompaniesLinks,temp1[0])
            
            temp1=link.split('nse/')
            temp2=temp1[1].split('.html')
            IndustryNames=np.append(IndustryNames,temp2[0])
            print(IndustryNames[len(IndustryNames)-1])
        i+=1
        print("Industries left",len(IndustryLinks)-i)
    except:
        print("Error at",link)
        

In [ ]:
i=0
for j in CompaniesLinks:
    try:
        temp=j.split('/')
        if(temp[5]!=''):
            IndustryNames[i]=IndustryNames[i]+' ('+temp[5]+')'
            CompanyNames=np.append(CompanyNames,temp[6])
        else:
            IndustryNames[i]=IndustryNames[i]+' ('+temp[5]+')'
            CompanyNames=np.append(CompanyNames,temp[7])
            
        row=np.array(terms(j))
            
        try:
            beta=np.append(beta,Beta(j))
        except:
            beta=np.append(beta,np.NaN)
                
        pe=np.append(pe,row[0])
        div=np.append(div,row[1])
        indpe=np.append(indpe,row[2])
        pb=np.append(pb,row[3])
        divy=np.append(divy,row[4])
        deliv1d=np.append(deliv1d,row[5])
        deliv3d=np.append(deliv3d,row[6])
        deliv5d=np.append(deliv5d,row[7])
        deliv8d=np.append(deliv8d,row[8])
        print(beta[len(beta)-1])
        print("Number of Companies Left:",(len(CompaniesLinks))-i)
        i+=1
    except:
        print(j)
        pass
    


In [ ]:
df['Industry']=IndustryNames
df['Company']=CompanyNames
df['Beta']=beta
df['P/E']=pe
df['Dividend(%)']=div
df['Industry P/E']=indpe
df['P/B']=pb
df['Dividend Yield(%)']=divy
df['Deliverables(1D)']=deliv1d
df['Deliverables(3D)']=deliv3d
df['Deliverables(5D)']=deliv5d
df['Deliverables(8D)']=deliv8d

In [ ]:
df

In [ ]:
df.to_csv('NseStockData.csv')